In [10]:
import os

from dotenv import load_dotenv
import httpx
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from fastapi import HTTPException

In [2]:
load_dotenv("../dev.env")

True

In [6]:
COLLECTION_NAME = "temp"
n_emb=384

In [7]:
client = QdrantClient(
            host=os.getenv("kb_host"), port=os.getenv("kb_port")
        )

In [11]:
if client.collection_exists(COLLECTION_NAME):
    raise HTTPException(status_code=400, detail="Collection already exists")

distance = "cosine"
# if not distance:
#     raise HTTPException(status_code=400, detail=f"Unknown distance metric: {collection.distance_metric}")

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=n_emb, distance=distance),
)


HTTPException: 400: Collection already exists

## Ingestion

In [20]:
data = [
    {
        "id": 1,
        "title": "Italian Pasta Recipe",
        "description": "Step by step recipe for making authentic Italian pasta with tomato sauce and basil.",
        "category": "food",
    },
    {
        "id": 2,
        "title": "Machine Learning Basics",
        "description": "Introduction to machine learning concepts including supervised and unsupervised learning.",
        "category": "technology",
    },
    {
        "id": 3,
        "title": "Paris Travel Guide",
        "description": "Best places to visit in Paris including the Eiffel Tower, Louvre Museum, and cafes.",
        "category": "travel",
    },
    {
        "id": 4,
        "title": "Personal Finance Tips",
        "description": "How to save money, budget effectively, and build long term wealth.",
        "category": "finance",
    },
    {
        "id": 5,
        "title": "Home Workout Routine",
        "description": "A 30 minute home workout plan that requires no equipment and improves fitness.",
        "category": "health",
    },
    {
        "id": 6,
        "title": "Python Async Programming",
        "description": "Guide to asynchronous programming in Python using asyncio and async await.",
        "category": "technology",
    },
    {
        "id": 7,
        "title": "Healthy Smoothie Ideas",
        "description": "Nutritious smoothie recipes with fruits, vegetables, and protein sources.",
        "category": "food",
    },
    {
        "id": 8,
        "title": "Stock Market Investing",
        "description": "Beginner guide to investing in the stock market and understanding risk.",
        "category": "finance",
    },
    {
        "id": 9,
        "title": "Hiking in the Alps",
        "description": "Experience hiking trails in the Alps with breathtaking mountain views.",
        "category": "travel",
    },
    {
        "id": 10,
        "title": "Football Training Drills",
        "description": "Effective football drills to improve speed, agility, and ball control.",
        "category": "sports",
    },
]


In [32]:
for idx, _data in enumerate(data):
    response = httpx.get(
        f"http://localhost:8000/v1/embedding/{_data['description']}"
    )
    embedding = response.json()["embedding"]

    client.upsert(
        collection_name=collection_name,
        points=[
            models.PointStruct(
                id=_data["id"],
                payload=_data,
                vector=embedding,
            ),
        ],
    )


KeyError: 'embedding'

In [33]:
response

<Response [404 Not Found]>